## Recommendation System
## Challenge 2
## Group 6
* ### Vrishmi Parikh (202318013) 
* ### Mahmood Topiwala (202318030)
* ### Anurag Shukla (202318039)
* ### Tanaz Pathan (202318056)

In [1]:
import pandas as pd
import numpy as np
import fastparquet
import surprise
from surprise.reader import Reader
from surprise.dataset import DatasetAutoFolds
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV
from surprise.accuracy import fcp, mae, mse, rmse
from surprise.prediction_algorithms.random_pred import NormalPredictor
from surprise.prediction_algorithms.baseline_only import BaselineOnly

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
songs = pd.read_csv("songs.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train.shape

(7377418, 6)

In [5]:
train["target"].isna().value_counts()

target
False    7377418
Name: count, dtype: int64

In [6]:
train.head(4)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1


In [7]:
train['msno'].nunique(), train['song_id'].nunique()

(30755, 359966)

In [8]:
train.groupby("msno")['msno'].value_counts().describe()

count    30755.000000
mean       239.877028
std        314.130788
min          1.000000
25%         27.000000
50%        128.000000
75%        340.000000
max       5819.000000
Name: count, dtype: float64

In [9]:
train.groupby("song_id")['song_id'].value_counts().describe()

count    359966.000000
mean         20.494763
std         182.237223
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max       13973.000000
Name: count, dtype: float64

In [10]:
s_lb = train.groupby("song_id")['song_id'].value_counts().quantile(.50)
s_ub = train.groupby("song_id")['song_id'].value_counts().quantile(.90)
print(s_lb, s_ub)

2.0 20.0


In [11]:
u_lb = train.groupby("msno")['msno'].value_counts().quantile(.50)
u_ub = train.groupby("msno")['msno'].value_counts().quantile(.90)
print(u_lb, u_ub)

128.0 615.0


In [12]:
_ = train.groupby("song_id").filter(lambda x: s_lb <= len(x) <= s_ub)
valid_ratings = _.groupby("msno").filter(lambda x: u_lb <= len(x) <= u_ub)

In [13]:
valid_ratings

,msno,song_id,source_system_tab,source_screen_name,source_type,target
10,uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=,t0aT90DlS1TGncgnKoL0SvfAWEr3Dl72QBVcokmKfLc=,my library,Local playlist more,local-library,1
77,yABGsFFqTctk/xnMj67nHmnisCwdue7XYxkYGOe9BGE=,RAVcXkNI6tqrsArzBxA9JcNnv/ftDwqDwBDaGqIzXyA=,my library,Local playlist more,local-library,1
78,yABGsFFqTctk/xnMj67nHmnisCwdue7XYxkYGOe9BGE=,ziioVtzot6FMViJgAMZliplvo7EmX3ecPuoZHXTr2Zo=,my library,Local playlist more,local-library,1
121,AXKTVBbdr2/z7m5WL1IXzlriNbun8B+7wM41MsWCYDw=,GopWs12UdPVQa3rEWVHv19FsoqHHsjo0il0Le89wN28=,discover,Discover Feature,album,0
122,AXKTVBbdr2/z7m5WL1IXzlriNbun8B+7wM41MsWCYDw=,8geiVKcfEd3jj9XiGb3hOVio3v0/BY8P29b2r3eo3Ps=,discover,Discover Feature,song-based-playlist,1
...,...,...,...,...,...,...
7377335,//4hBneqk/4/TtgL1XXQ+eKx7fJTeSvSNt0ktxjSIYE=,cjFcw6k2T1Loci4oAHn/Ob5b29zOSUqlTC6I1bodGF4=,search,Search,album,0
7377336,//4hBneqk/4/TtgL1XXQ+eKx7fJTeSvSNt0ktxjSIYE=,ZvQjd+l83+MnLmFIdIU264CfrtUBcKxOpr3UBBYp1Bk=,search,Search,album,1
7377337,//4hBneqk/4/TtgL1XXQ+eKx7fJTeSvSNt0ktxjSIYE=,9C/pDOdVTSuPd3uQFrlCRk1Ljk5oNTMSLklUU4lS//I=,search,Search,album,0
7377338,//4hBneqk/4/TtgL1XXQ+eKx7fJTeSvSNt0ktxjSIYE=,yeWoUcjnEnE3BI+bq0Gi94azpn5t6vBVThbbR/veh3g=,search,Search,album,0


In [14]:
valid_train, valid_test = train_test_split(
    valid_ratings,
    train_size=0.8,
    stratify=valid_ratings["msno"],
    random_state=42,
)

In [15]:
valid_train['msno'].nunique(), valid_train['song_id'].nunique()

(1480, 118079)

In [16]:
reader = Reader(rating_scale=(0, 1))

data_train = DatasetAutoFolds(
    reader=reader,
    df=valid_train[["msno", "song_id", "target"]],
).build_full_trainset()
_raw_data_train = DatasetAutoFolds(
    reader=reader, df=valid_train[["msno", "song_id", "target"]]
)

In [17]:
param_grid = {
    "n_factors": [50, 100, 150],
    "biased": [True],
    "lr_all": [0.001, 0.005, 0.01, 0.015],
    "reg_all": [0.005, 0.01, 0.05, 0.1],
    "n_epochs": [10],
}

gs = GridSearchCV(
    SVD,
    param_grid,
    measures=["mae", "mse"],
    cv=5,
    refit=True,
    n_jobs=-1,
    joblib_verbose=5,
)

In [18]:
gs.fit(_raw_data_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   39.6s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.3min finished


In [19]:
df = pd.DataFrame(gs.cv_results)
df_final = df[
    [
        "mean_test_mae",
        "std_test_mae",
        "mean_test_mse",
        "std_test_mse",
        "params",
    ]
]
df_final.sort_values("mean_test_mse", ascending=True).head(10)

,mean_test_mae,std_test_mae,mean_test_mse,std_test_mse,params
7,0.411953,0.000534,0.205012,0.000672,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."
6,0.408916,0.000488,0.205698,0.000683,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."
11,0.411375,0.000572,0.205948,0.000728,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."
23,0.412354,0.000498,0.205980,0.000600,"{'n_factors': 100, 'biased': True, 'lr_all': 0..."
27,0.411626,0.000574,0.206358,0.000721,"{'n_factors': 100, 'biased': True, 'lr_all': 0..."
10,0.408222,0.000478,0.206589,0.000700,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."
43,0.411957,0.000511,0.206788,0.000662,"{'n_factors': 150, 'biased': True, 'lr_all': 0..."
39,0.412690,0.000406,0.206889,0.000585,"{'n_factors': 150, 'biased': True, 'lr_all': 0..."
5,0.406655,0.000464,0.207226,0.000601,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."
4,0.406154,0.000505,0.207337,0.000690,"{'n_factors': 50, 'biased': True, 'lr_all': 0...."


In [20]:
gs.best_params, gs.best_score

({'mae': {'n_factors': 50,
   'biased': True,
   'lr_all': 0.015,
   'reg_all': 0.005,
   'n_epochs': 10},
  'mse': {'n_factors': 50,
   'biased': True,
   'lr_all': 0.005,
   'reg_all': 0.1,
   'n_epochs': 10}},
 {'mae': 0.40531259467233915, 'mse': 0.20501227722824047})

In [21]:
#! Optimmized for MSE

_data_test = (
    DatasetAutoFolds(
        reader=reader,
        df=valid_test[["msno", "song_id", "target"]],
    )
    .build_full_trainset()
    .build_testset()
)

gs_r = gs.test(_data_test)
print(f" FCP: {fcp(gs_r,verbose=False)}")
print(f" MAE: {mae(gs_r,verbose=False)}")
print(f" MSE: {mse(gs_r,verbose=False)}")
print(f"RMSE: {rmse(gs_r,verbose=False)}")

 FCP: 0.4897747905125702
 MAE: 0.4049779672132088
 MSE: 0.2131150180076361
RMSE: 0.46164382158503553


In [26]:
def get_metrics(usr):
        tdf = valid_test[valid_test["msno"] == usr]
        subs = []

        for i in range(len(tdf)):
                msno_, song_id_, trgt = tdf.iloc[i][["msno", "song_id", "target"]]
                pred = gs.predict(msno_, song_id_).est
                subs.append((trgt, pred))

        m = np.mean([i[1] for i in subs])
        final_subs = []
        for i in subs:
            if i[1] > m:
                final_subs.append((i[0], 1))
            else:
                final_subs.append((i[0], 0))

        n = len(final_subs)
        not_accurate_pre = 0
        for i in final_subs:
            not_accurate_pre += (i[0] ^ i[1])

        
        not_accurate_re = 0
        usr_pref = [i for i in final_subs if i[0] == 1]
        npref = len(usr_pref)
        for i in usr_pref:
            not_accurate_re += (i[0] ^ i[1])
        
        print(f"User: {usr}")
        print("Precision: ", (n-not_accurate_pre)/n)
        if npref == 0:
            print("Recall:    ", "No items liked by user")
        else:
            print("Recall:    ", (npref-not_accurate_re)/npref)
        print("\n==================================\n")

        
usr = "vJW9mmXL4ScyBC3Ubt/iHoQv6I5/Aw/LdWeQVooMvt0="
get_metrics(usr)

User: vJW9mmXL4ScyBC3Ubt/iHoQv6I5/Aw/LdWeQVooMvt0=
Precision:  0.7105263157894737
Recall:     0.8571428571428571


